In [1]:
from __future__ import annotations
from pathlib import Path
from typing import Dict, Tuple, List
import os
import json
import pandas as pd
import numpy as np
from loguru import logger

In [2]:
BASE_DIR = os.getcwd()
HOME_DIR = os.path.dirname(BASE_DIR)
old_data_dates = os.listdir(os.path.join(HOME_DIR, "hl-node-trades"))

# --- Config/paths ---
DATA_DIR = Path(os.path.join(HOME_DIR, "data"))
DATA_DIR.mkdir(parents=True, exist_ok=True)

WALLETS_CSV = DATA_DIR / "wallet_db.csv"

In [3]:
# --- Wallet DB helpers ---
def load_wallet_db(csv_path: Path = WALLETS_CSV) -> Tuple[Dict[str, int], int]:
    """
    Load wallets from CSV into a dict {wallet: wallet_id}, return dict and next_id.
    If file doesn't exist, start fresh at 1.
    """
    mapping: Dict[str, int] = {}
    next_id = 1
    if csv_path.exists():
        df = pd.read_csv(csv_path, dtype={"wallet_id": "uint32", "wallet": "string"})
        if not df.empty:
            for wid, wal in zip(df["wallet_id"].astype("uint32"), df["wallet"].astype("string")):
                mapping[str(wal)] = int(wid)
            next_id = int(df["wallet_id"].max()) + 1
    else:
        csv_path.parent.mkdir(parents=True, exist_ok=True)
        pd.DataFrame(columns=["wallet_id", "wallet"]).to_csv(csv_path, index=False)
    return mapping, next_id

def append_wallet(csv_path: Path, wallet: str, wallet_id: int) -> None:
    """Append a single wallet row to the CSV."""
    pd.DataFrame([{"wallet_id": wallet_id, "wallet": wallet}]).to_csv(
        csv_path, mode="a", header=False, index=False
    )

def get_wallet_id(wallet: str, mapping: Dict[str, int], next_id_ref: List[int], csv_path: Path) -> int:
    """
    Return wallet_id for wallet, creating a new id if needed.
    next_id_ref is a single-item list to allow in-place increment.
    """
    w = str(wallet)
    wid = mapping.get(w)
    if wid is not None:
        return wid
    wid = next_id_ref[0]
    mapping[w] = wid
    next_id_ref[0] += 1
    append_wallet(csv_path, w, wid)
    return wid

# --- ETL helpers ---
def _infer_is_ask(trade: dict, idx_in_side_info: int) -> bool:
    """
    Decide if the row belongs to the ask side.
    Heuristic:
      1) If top-level 'side' is present ('A'/'B'), we tag A as asks, B as bids.
      2) Otherwise fall back to index parity inside side_info: even->A(ask), odd->B(bid).
    Adjust here if your ground truth differs.
    """
    is_ask = (idx_in_side_info % 2 == 0)  # even -> "A" (ask), odd -> "B" (bid)
    return bool(is_ask)

def retrieve_data(file_path: Path, wallet_map: Dict[str, int], next_id_ref: List[int], wallets_csv: Path = WALLETS_CSV) -> pd.DataFrame:
    """
    Read a newline-delimited JSON file of trades and produce a normalized DataFrame
    for later partitioned saving.
    Output columns: coin, price, size, time, is_ask, wallet_id
    """
    records = []
    with open(file_path) as f:
        append = records.append
        for line in f:
            if not line.strip():
                continue
            trade = json.loads(line)

            side_info_list = trade.get("side_info") or []
            for idx, side_info in enumerate(side_info_list):
                user = side_info.get("user")
                if user is None:
                    continue

                wallet_id = get_wallet_id(user, wallet_map, next_id_ref, wallets_csv)

                px = trade.get("px")
                sz = trade.get("sz")
                # skip malformed
                if px is None or sz is None:
                    continue

                append(
                    {
                        "coin": trade.get("coin"),
                        "price": float(px),
                        "size": float(sz),
                        "time": trade.get("time"),
                        "is_ask": _infer_is_ask(trade, idx),
                        "wallet_id": wallet_id,
                    }
                )

    df = pd.DataFrame.from_records(records)
    if df.empty:
        return df

    # Types & cleaning
    df["time"] = pd.to_datetime(df["time"], errors="coerce")
    df = df.dropna(subset=["time"])
    # enforce dtypes
    df["price"] = df["price"].astype("float32")
    df["size"] = df["size"].astype("float32")
    df["is_ask"] = df["is_ask"].astype("bool")
    df["wallet_id"] = df["wallet_id"].astype("uint32")

    return df[["coin", "price", "size", "time", "is_ask", "wallet_id"]]

def _target_path_for(coin: str, dt: pd.Timestamp) -> Path:
    return DATA_DIR / str(coin) / f"{dt.date()}.parquet"

def _write_daily_parquet(target: Path, df_day: pd.DataFrame) -> None:
    """
    Write/merge the daily file. If target exists, read, concat, de-dup, sort, write.
    We de-dup on [time, wallet_id, price, size, is_ask] as a reasonable row identity.
    """
    target.parent.mkdir(parents=True, exist_ok=True)

    # Keep only required columns & types
    cols = ["price", "size", "time", "is_ask", "wallet_id"]
    df_day = df_day[cols].copy()

    if target.exists():
        try:
            old = pd.read_parquet(target, engine="pyarrow")
            # Cast to same dtypes to avoid upcasting surprises
            old["price"] = old["price"].astype("float32")
            old["size"] = old["size"].astype("float32")
            old["time"] = pd.to_datetime(old["time"], errors="coerce")
            old["is_ask"] = old["is_ask"].astype("bool")
            old["wallet_id"] = old["wallet_id"].astype("uint32")
            df_day = pd.concat([old, df_day], ignore_index=True)
        except Exception as e:
            logger.warning(f"Failed to read existing parquet {target}: {e}. Overwriting.")

    df_day = df_day.dropna(subset=["time"]).drop_duplicates(
        subset=["time", "wallet_id", "price", "size", "is_ask"], keep="last"
    )
    df_day = df_day.sort_values("time")
    df_day.to_parquet(target, index=False, engine="pyarrow", compression="snappy")

def save_partitioned(df: pd.DataFrame) -> None:
    """
    Save rows to data/<coin>/<YYYY-MM-DD>.parquet, merging per-day files if present.
    """
    if df.empty:
        logger.warning("No data to save.")
        return

    # Add date for grouping
    df = df.copy()
    df["date"] = df["time"].dt.date

    # Group by coin/date
    for (coin, day), g in df.groupby(["coin", "date"], sort=False):
        if pd.isna(coin) or coin == "":
            logger.warning("Skipping rows with empty coin.")
            continue
        target = DATA_DIR / str(coin) / f"{day}.parquet"
        _write_daily_parquet(target, g)

    logger.info("Data has been saved successfully.")

In [4]:
old_data_folders = os.listdir(os.path.join(HOME_DIR, "hl-node-trades"))

wallet_map, next_id = load_wallet_db()
next_id_ref = [next_id]  # mutable holder


for i, date in enumerate(old_data_folders):
    hour_file_names = os.listdir(os.path.join(HOME_DIR, "hl-node-trades", date))

    for file_name in hour_file_names:
        file_full_path = os.path.join(HOME_DIR, "hl-node-trades", date, file_name)

        logger.info(f"{file_full_path} is processing")
        df = retrieve_data(Path(file_full_path), wallet_map, next_id_ref, WALLETS_CSV)
        save_partitioned(df)

    logger.info(f"Processed {i} out of {len(old_data_folders)}")

2025-08-20 17:34:18.447 | INFO     | __main__:<module>:13 - /home/debian/hl-node-trades/20250322/10.json is processing
2025-08-20 17:34:19.375 | INFO     | __main__:save_partitioned:160 - Data has been saved successfully.
2025-08-20 17:34:19.377 | INFO     | __main__:<module>:13 - /home/debian/hl-node-trades/20250322/14.json is processing
2025-08-20 17:34:21.845 | INFO     | __main__:save_partitioned:160 - Data has been saved successfully.
2025-08-20 17:34:21.851 | INFO     | __main__:<module>:13 - /home/debian/hl-node-trades/20250322/12.json is processing
2025-08-20 17:34:24.425 | INFO     | __main__:save_partitioned:160 - Data has been saved successfully.
2025-08-20 17:34:24.428 | INFO     | __main__:<module>:13 - /home/debian/hl-node-trades/20250322/19.json is processing
2025-08-20 17:34:26.498 | INFO     | __main__:save_partitioned:160 - Data has been saved successfully.
2025-08-20 17:34:26.501 | INFO     | __main__:<module>:13 - /home/debian/hl-node-trades/20250322/21.json is proc

In [14]:
!pip install pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 43.7 MB/s eta 0:00:0000:0100:01


In [18]:
import sys

print(sys.getsizeof(20))         # int: 28
print(sys.getsizeof(3.14))      # float: 24
print(sys.getsizeof(""))        # str: 49
print(sys.getsizeof("a"))       # str: 50
print(sys.getsizeof("0x"*21))     # str: 52

28
24
49
50
91
80


In [ ]:
def load_wallet_db(csv_path: Path) -> Tuple[Dict[str, int], int]:
    """
    Load wallets from CSV into a dict {wallet: wallet_id}, return dict and next_id.
    If file doesn't exist, start fresh at 1.
    """
    mapping = {}
    next_id = 1
    if csv_path.exists():
        df = pd.read_csv(csv_path, dtype={"wallet_id": "uint32", "wallet": "string"})
        # handle empties gracefully
        if not df.empty:
            # ensure unique and correct mapping
            for wid, wal in zip(df["wallet_id"].astype("uint32"), df["wallet"].astype("string")):
                mapping[str(wal)] = int(wid)
            next_id = int(df["wallet_id"].max()) + 1
    else:
        # create file with header
        csv_path.parent.mkdir(parents=True, exist_ok=True)
        pd.DataFrame(columns=["wallet_id", "wallet"]).to_csv(csv_path, index=False)
    return mapping, next_id

def append_wallet(csv_path: Path, wallet: str, wallet_id: int) -> None:
    """Append a single wallet row to the CSV."""
    # minimal IO per new wallet
    pd.DataFrame(
        [{"wallet_id": wallet_id, "wallet": wallet}]
    ).to_csv(csv_path, mode="a", header=False, index=False)

def get_wallet_id(wallet: str, mapping: Dict[str, int], next_id_ref: list, csv_path: Path) -> int:
    """
    Return wallet_id for wallet, creating a new id if needed.
    next_id_ref is a single-item list to allow in-place increment.
    """
    w = str(wallet)
    wid = mapping.get(w)
    if wid is not None:
        return wid
    # assign new
    wid = next_id_ref[0]
    mapping[w] = wid
    next_id_ref[0] += 1
    append_wallet(csv_path, w, wid)
    return wid


def retrieve_data(file_path):
    records = []

    with open(file_path) as f:
        append = records.append
        for line in f:
            trade = json.loads(line)
            
            side_info_list = trade.get("side_info") or []
            for idx, side_info in enumerate(side_info_list):
                side = "A" if idx % 2 == 0 else "B"

                user = side_info.get("user")
                wallet_id = get_wallet_id(user, wallet_map, next_id_ref, WALLETS_CSV)
                
                append(
                    {
                        "coin": trade.get("coin"),
                        "price": float(trade.get("px")),
                        "size": float(trade.get("sz")),
                        "time": trade.get("time"),
                        "side": side,
                        "wallet": wallet_id,
                    }
                )

    df = pd.DataFrame.from_records(records)
    df["time"] = pd.to_datetime(df["time"], errors="coerce")
    return df


def filter_data(df, coin):
    return df[df["coin"] == coin]


def save_data(df, output_file):
    # Write to disk
    if not df.empty:
        df.to_parquet(output_file, index=False, engine='pyarrow', compression='snappy', append=os.path.exists(output_file))
        logger.info("Data has been saved successfully.")
    else:
        logger.warnning("No data to save.")


In [8]:
pd.read_csv(WALLETS_CSV, index_col="wallet_id")

,wallet
wallet_id,
1,0x31ca8395cf837de08b24da3f660e77761dfb974b
2,0x995f7741ab2d2d03dd1502f45e63e0adfda44dcc
3,0x223537ac9a856c31f4043e86ced86bb29f06653e
4,0x59891acadc0f8b2bcab10c043ac5fc63628b1716
5,0x5887de8d37c9c2550a4d0b86127c43b2e1904545
...,...
9052,0xbb5048a65be6366beb86e06b21fa3018bb99340d
9053,0x293e5bc2a68027635a84df9966ea2f487906f5f3
9054,0xcb8d02079049943ea129ac10bbb51f5f9beb1128


In [13]:
pd.read_parquet(DATA_DIR / "ETH" / "2025-03-22.parquet")

,price,size,time,is_ask,wallet_id
0,1996.500000,0.0062,2025-03-22 10:50:22.049676014,True,81
1,1996.500000,0.0062,2025-03-22 10:50:22.049676014,True,82
2,1996.500000,0.0075,2025-03-22 10:50:22.049676014,True,81
3,1996.500000,0.0075,2025-03-22 10:50:22.049676014,True,82
4,1996.500000,8.0000,2025-03-22 10:50:22.049676014,True,83
...,...,...,...,...,...
90140,1979.599976,0.2527,2025-03-22 23:59:57.997237907,True,827
90141,1979.599976,1.7675,2025-03-22 23:59:57.997237907,True,826
90142,1979.599976,2.2007,2025-03-22 23:59:57.997237907,True,39
90143,1979.599976,0.0088,2025-03-22 23:59:59.328207652,False,81


In [8]:
old_data_folders = os.listdir(os.path.join(HOME_DIR, "hl-node-trades"))

wallet_map, next_id = load_wallet_db()
next_id_ref = [next_id]  # mutable holder

In [37]:
df_fun = retrieve_data(Path(HOME_DIR) / "hl-node-trades" / "20250322" / "10.json", wallet_map, next_id_ref, WALLETS_CSV)

In [14]:
file_path = Path(HOME_DIR) / "hl-node-trades" / "20250322" / "10.json"

In [24]:
records = []
with open(file_path) as f:
    append = records.append
    for line in f:
        trade = json.loads(line)
        
        side_info_list = trade.get("side_info")
        if len(side_info_list) != 2:
            print(len(side_info_list))
        
        for side_info, side in list(zip(side_info_list, ["A", "B"])):
            user = side_info.get("user")
            wallet_id = get_wallet_id(user, wallet_map, next_id_ref, WALLETS_CSV)
            
            append(
                {
                    "coin": trade.get("coin"),
                    "price": float(trade.get("px")),
                    "size": float(trade.get("sz")),
                    "time": trade.get("time"),
                    "side": side,
                    "wallet": wallet_id,
                }
            )

df = pd.DataFrame.from_records(records)
df["time"] = pd.to_datetime(df["time"], errors="coerce")
# return df

In [26]:
trade.get("side")

'A'

In [38]:
df_fun

,coin,price,size,time,is_ask,wallet_id
0,CRV,0.46747,4102.899902,2025-03-22 10:48:33.216798262,True,1
1,CRV,0.46747,4102.899902,2025-03-22 10:48:33.216798262,False,2
2,CRV,0.46732,2006.199951,2025-03-22 10:48:33.216798262,True,3
3,CRV,0.46732,2006.199951,2025-03-22 10:48:33.216798262,False,2
4,CRV,0.46731,4012.300049,2025-03-22 10:48:33.216798262,True,4
...,...,...,...,...,...,...
20683,BTC,84385.00000,0.030000,2025-03-22 10:59:59.719025144,False,82
20684,BADGER,1.69310,12.300000,2025-03-22 10:59:59.719025144,True,39
20685,BADGER,1.69310,12.300000,2025-03-22 10:59:59.719025144,False,1
20686,GAS,2.97430,13.500000,2025-03-22 10:59:59.719025144,True,1


In [21]:
df

,coin,price,size,time,side,wallet
0,CRV,0.46747,4102.90,2025-03-22 10:48:33.216798262,A,1
1,CRV,0.46747,4102.90,2025-03-22 10:48:33.216798262,B,2
2,CRV,0.46732,2006.20,2025-03-22 10:48:33.216798262,A,3
3,CRV,0.46732,2006.20,2025-03-22 10:48:33.216798262,B,2
4,CRV,0.46731,4012.30,2025-03-22 10:48:33.216798262,A,4
...,...,...,...,...,...,...
20683,BTC,84385.00000,0.03,2025-03-22 10:59:59.719025144,B,82
20684,BADGER,1.69310,12.30,2025-03-22 10:59:59.719025144,A,39
20685,BADGER,1.69310,12.30,2025-03-22 10:59:59.719025144,B,1
20686,GAS,2.97430,13.50,2025-03-22 10:59:59.719025144,A,1


In [27]:
trade

{'coin': 'GAS',
 'side': 'A',
 'time': '2025-03-22T10:59:59.719025144',
 'px': '2.9743',
 'sz': '13.5',
 'hash': '0x3ff61a30b2673cef41ba04200bb69d020143002ac566845c745180be58eb56fc',
 'trade_dir_override': 'Na',
 'side_info': [{'user': '0x31ca8395cf837de08b24da3f660e77761dfb974b',
   'start_pos': '-61673.4',
   'oid': 81245875058,
   'twap_id': None,
   'cloid': None},
  {'user': '0x010461c14e146ac35fe42271bdc1134ee31c703a',
   'start_pos': '63854.8',
   'oid': 81245877405,
   'twap_id': None,
   'cloid': None}]}